In [1]:
%cd /Users/trancatkhanh/Downloads/GAME

/Users/trancatkhanh/Downloads/GAME


In [2]:
import numpy as np
from base.MachiKoro.env import *
#0 Array bias trong mỗi trận
#1 Array bias sau 100k trận 
#2 Lưu số trận 
#3 lưu hệ số đã dùng 
#4 Tỷ lệ thắng khi đấu 10k trận 
#5 Trigger để train 100k trận 
#6 Đếm số lần yếu liên tiếp
#7 Lưu array cho policy yếu 
#8 State của trận đấu
#9 Action của trận đấu
#10 Reward của trận đấu
#11 Số trận lưu state
#12 Trigger policy yếu
#13 Lưu array của decision tree
#14 Lưu giá trị để chia Node (TBC active win -TBC active lose)
#15 Lưu thứ tự node của decision tree
perx = [np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,getActionSize())),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,2)),np.zeros((1,2)),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((200000,getStateSize())),np.zeros((200000,1)),np.zeros((200000,1)),np.zeros((1,1)),np.zeros((1,2)),np.array([np.random.rand(getStateSize())-0.55 for _ in range(4)]),np.zeros((200000,4)),np.zeros((4,1))]
                         #0                                                      #1                   #2              #3                                                            #4              #5               #6                #7                                                               #8                                   #9               #10                   #11           #12              #13                                                             #14              #15

In [10]:
(perx[3])

array([[0.84826405, 0.80801367, 0.86932802, 0.86610927, 0.86565106,
        0.87811492, 0.85736194, 0.82135671, 0.85909471, 0.84328403,
        0.85790064, 0.89995426, 0.84835515, 0.80948097, 0.86466602,
        0.82410056, 0.84982254, 0.87492908, 0.84781929, 0.87859223,
        0.86999008, 0.8382613 , 0.82007808, 0.8438337 , 0.85931125,
        0.86147889, 0.84181652, 0.82926894, 0.84066271, 0.83554712,
        0.8725738 , 0.81538386, 0.84443684, 0.85203694, 0.95422792,
        1.        , 0.82912976, 0.83440044, 0.89140411, 0.9152381 ,
        0.78156193, 0.77978005, 0.80787309, 0.75286951, 0.79912502,
        0.79930468, 0.82916356, 0.82300684, 0.80715687, 0.96634157,
        0.97772353, 0.99777836, 0.95595222, 0.27310861]])

# Hàm xây cây 


In [4]:
import numpy as np
@njit
def Build_decision_tree(per):
    # replace -1 with 1 and 0
    for i in range(len(per[10])):
        if per[10][i] == -1:
            for j in range(i,len(per[10])):
                if per[10][j] != -1:
                    per[10][i] = per[10][j]
                    break
    # replace -1 with 0
    for i in range(len(per[10])):
        if per[10][i] == -1:
            per[10][i] = 0
            
    # Calculate the active match and non-active match
    for j in range(4):
        for i in range(len(per[8])):
            per[14][i][j] += (per[8][i]@per[13][j])

    #Calculate the active win and acitve lose
    for j in range(4):
        for i in range(len(per[8])):
            if per[14][i][j] > 1 and per[10][i] == 1:
                per[15][j] += 1
            if per[14][i][j] > 1 and per[10][i] == 0:
                per[15][j] -= 1 
    # Find build the decision tree
    # Node 1 np.min(score)
    # Node 2 second min score
    # Node 3 third min score
    # Node 4 max score
    return np.min(per[15])
@njit 
def ChooseActionDecisionTree(state,per):
    if per[13]

# Agent Hoàn Chỉnh

In [3]:
import numpy as np
@njit
def check_min(state, reward):
    just = np.zeros((200000,2))
    for i in range(len(reward)):
        if reward[i] == -1:
            for j in range(i,len(reward)):
                if reward[j] != -1:
                    reward[i] = reward[j]
                    break
    # replace -1 with 0
    for i in range(len(reward)):
        if reward[i] == -1:
            reward[i] = 0
    # np.mean each array in the state and the corresponding reward
    for i in range(len(reward)):
        just[i][0] = np.mean(state[i])
        just[i][1] = (reward[i][0])
    # count the number of times each value of the state appears and the reward of each value of the state
    count = np.zeros((200000,3))
    for i in range(len(reward)):
        for j in range(len(count)):
            if just[i][0] == count[j][0]:
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
            if count[j][0] == 0:
                count[j][0] = just[i][0]
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
    # find the index value the lowest rate of reward has the highest number of times appear
    min = 0
    for i in range(len(count)):
        if count[i][1] > count[min][1]:
            min = i
    for i in range(len(count)):
        if count[i][1] == count[min][1] and count[i][2] < count[min][2]:
            min = i

    return count[min][0]
@njit
def check_weaker(min,x):
    if x == min:
        return 1
    else:
        return 0

@njit()
def p_bias(state,per):
    # Chuyển Mode gồm 3 mode mode 0 - train bias , mode 1 - test bias , mode 2 - train policy yếu ,mode 3 - test policy yếu,mode 4 - lưu state
    if per[4][0][0] > per[5][0][1] and per[4][0][1] == 10000 and per[5][0][0] == 1:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      # Lưu lại kết quả thắng
      per[5][0][1] = 0 
      per[5][0][1] += (per[4][0][0])
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Reset số lần yếu liên tiếp
      per[6][0][0] = 0 
    elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 10000 and per[5][0][0] == 1:
      per[6][0][0] +=1 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 

    # Bắt đầu mode lưu state
    if per[6][0][0] == 3 and per[11][0][0] < 200000 :
      # Chuyển sang lưu state 
      per[5][0][0] = 4
      per[2][0][0] = 0
      # Rest số trận thắng và số trận đấu 
      per[4][0][1] = 0 
      per[4][0][0] = 0
      
    # Hết mode lưu state chuyển sang train policy cho các arr
    if per[5][0][0] == 4 and per[11][0][0] == 200000:
      per[5][0][0] = 2

    # Bất đầu mode train và test policy arr
    if per[11][0][0] == 200000:
      if per[4][0][0] > per[5][0][1] and per[4][0][1] == 10000 and per[5][0][0] == 3:
        per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
        # Lưu lại kết quả thắng
        per[5][0][1] = 0 
        per[5][0][1] += (per[4][0][0])
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))

      elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 10000 and per[5][0][0] == 3:
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
      
    # reset per[1][0] sau 100k trận
    if int(per[2][0][0]) == 10000 and per[11][0][0] < 200000:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 1
    # reset per[1][0] sau 10k trận
    if int(per[2][0][0]) == 10000 and per[11][0][0] == 200000:
      per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 3
      
    if per[11][0][0] == 200000 and per[12][0][0] == 0:
      per[12][0][0] += check_min(per[8], per[10])

    # Chế độ train bias
    if int(per[2][0][0]) < 10000 and per[5][0][0] == 0  :
      actions = getValidActions(state)
      actions *= per[0][0]
      action = np.argmax(actions)
      if getReward(state) == 1 :
          per[1] += per[0][0]
      if getReward(state) == 0:
          per[0][0] = np.random.rand(getActionSize())

    # Chế độ test bias
    elif int(per[2][0][0]) <10000  and per[5][0][0] == 1 : 
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0])    
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  

    # Chế độ train policy arr
    elif int(per[2][0][0]) < 10000 and per[5][0][0] == 2 and per[11][0][0] == 200000 :
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      Check_yeu = check_weaker(per[12][0][0],np.mean(state))
      if Check_yeu == 1:
        per[12][0][1] = Check_yeu
        actions *= per[0][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
        
      if getReward(state) == 1 and per[12][0][1] == 1:
        per[1] += per[0][0]
        per[12][0][1] = 0 
      if getReward(state) == 0 and per[12][0][1] == 1:
        per[0][0] = np.random.rand(getActionSize())
        per[12][0][1] = 0 

    # chế độ test policy yếu
    elif int(per[2][0][0]) <10000 and  per[5][0][0] == 3 and per[11][0][0] == 200000:
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      if check_weaker(per[12][0][0],np.mean(state)) == 1:
        actions *= per[7][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  
    

    # Báo chuyển sang chế độ lưu state và action
    if per[5][0][0] == 4 and per[11][0][0] < 200000:
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0]) 


    # Lưu state và action
    if getReward(state) != -2 and per[11][0][0] < 200000 and per[5][0][0] == 4:
        for i in range(getStateSize()):
          per[8][int(per[11][0][0])][i] += state[i]        
        per[9][int(per[11][0][0])][0] += int(action)
        per[10][int(per[11][0][0])][0] += int(getReward(state))
        per[11][0][0] += 1

    # đếm số ván đấu 
    if getReward(state) != -1 :
        per[2][0][0] += 1
    return action,per

win, bias = numba_main_2(p_bias, 200000,perx,0)

# Test lại tỷ lệ thắng của level 1 game MachiKoro

**Khi chưa khắc phục policy yếu :24551** 


**Khi đã khắc phục policy yếu :27961**

In [4]:
@njit()
def test(state,per):
    actions = getValidActions(state)
    # Check xem policy có yếu không 
    if check_weaker(per[12][0][0],np.mean(state)) == 1:
      actions *= per[7][0]
      action = np.argmax(actions)
    else:
      actions *= per[3][0]
      action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(test, 10000,perx,1)
print(win)

1761


# Tạo cây rỗng ==> Tạo các tính chất trước 

In [7]:
%cd /Users/trancatkhanh/Downloads/GAME
import numpy as np
from base.MachiKoro.env import *
per_state= [np.zeros((50000,getStateSize())),np.zeros((50000,1)),np.zeros((1,1))]

@njit()
def tao_arr(state,per):
    actions = getValidActions(state)
    # Check xem policy có yếu không 
    l = np.where(actions ==1)[0]
    action = np.random.choice(l)
    if per[2][0][0] < 50000:
        for i in range(getStateSize()):
            per[0][int(per[2][0][0])][i] += state[i]
        per[1][int(per[2][0][0])][0] += getReward(state)
        per[2][0][0]+=1 
    return action,per
@njit
win, bias = numba_main_2(tao_arr, 1000,per_state,0)
print(win)

/Users/trancatkhanh/Downloads/GAME
271


In [6]:
for i in range(len(per_state[1])):
    if per_state[1][i] == -1:
        for j in range(i,len(per_state[1])):
            if per_state[1][j] != -1:
                per_state[1][i] = per_state[1][j]
                break
# replace -1 with 0
for i in range(len(per_state[1])):
    if per_state[1][i] == -1:
        per_state[1][i] = 0
np.where(per_state[1]==1)[0]

array([  131,   132,   133, ..., 49775, 49776, 49777])

In [11]:
l@per_state[0][1]

54.41581469623884

In [72]:
l = np.array([np.random.rand(getStateSize())-0.55 for _ in range(4)])
k = np.zeros((50000,4))
# save all the value of l @ each array of per_state[0]
for j in range(4):
    for i in range(len(per_state[0])):
        k[i][j] += (per_state[0][i]@l[j])
score = np.zeros((4,1))
# if k[i][j] > 1 and per_state[1][i] == 1 then score[j] += 1 
# if k[i][j] > 1 and per_state[1][i] == 0 then score[j] -= 1
for j in range(4):
    for i in range(len(per_state[0])):
        if k[i][j] > 1 and per_state[1][i] == 1:
            score[j] += 1
        if k[i][j] > 1 and per_state[1][i] == 0:
            score[j] -= 1 


In [74]:
score

array([[ -42.],
       [ -11.],
       [ -57.],
       [-329.]])